# 검색
- counts의 field로 정렬
- 상위 10개만 가져옴
- 오타 수정(보정) 가능하게 설정
- 인덱싱: 자소분리 토크나이저

<br>

In [1]:
import requests
import json



# 응답을 프린트하는 method
def print_result(parsed_data):
    for data in parsed_data['hits']['hits']:
        print(data['_source']['name'])

# 응답을 받아오는 method
def get_response(body):
    response = requests.get(URL, 
                        data= json.dumps(body),
                        headers={"Content-Type": "application/json"}) 

    if(response.status_code==200):
        data = response.text
        parsed_data = json.loads(data)
    else:
        print(response.status_code)
        print(response.json())
    return parsed_data

In [2]:
URL = 'http://106.10.42.5:9200/autocomplete_jaso/_search' 
body = {
            "sort":[
                {"counts.default":{"order":"desc"}},
                "_score"
            ],
            "size":10,
            "query": {
                "match": {
                  "name": {
                      "query": "검색어",
                      "fuzziness":"AUTO"
                  }
                }
              }
            }

참고: 구분선 위의 코드를 실행해야 아래의 코드가 실행됩니다.
<hr>
# 1) 지정하지 않았을 때
### "롱"으로 검색

In [5]:
body['query']['match']['name']['query'] = "롱"
parsed_data = get_response(body)

#### 결과
- "롱"패딩 등의 단어가 검색됨
- "공"기 청정기도 검색됨

In [6]:
print_result(parsed_data)

롱패딩
키즈 롱패딩
롱원피스
디스커버리 롱패딩
롱코트
롱부츠
롱무스탕
공기청정기
여성 롱패딩
롱스커트


### "ㅍㄷ"를 검색

In [7]:
body['query']['match']['name']['query'] = "ㅍㄷ"
parsed_data = get_response(body)

#### 결과
- "ㅍㄷ"이 포함되는 단어가 검색됨
- 오타 수정의 여지는 없는 것 같음

In [8]:
print_result(parsed_data)

패딩
패딩조끼
패딩부츠
패딩우주복
패딩슬립온
패딩점퍼
패딩바지
패딩코트
펜디가방
패딩베스트


### "패딩"을 "페딩"으로 오타낸 것으로 가정

In [13]:
body['query']['match']['name']['query'] = "페딩"
parsed_data = get_response(body)

#### 결과
- "패딩"으로 수정되어 검색됨을 알 수 있음

In [14]:
print_result(parsed_data)

패딩
패딩조끼
여성 패딩
롱 패딩
k2 패딩
패딩부츠
패딩코트
아이더 패딩
웰론 패딩
뉴발란스 패딩


### 한/영 오타를 낸 것으로 가정

In [9]:
body['query']['match']['name']['query'] = "voeld"
parsed_data = get_response(body)

#### 결과
- "패딩"을 검색한 것처럼 검색이 됨

In [10]:
print_result(parsed_data)

패딩
패딩조끼
디스커버리 패딩
남자 패딩
아이더 패딩
키즈 패딩
노스페이스 패딩
여성 패딩
남자 패딩코트
k2 패딩


- 실제로 "패딩"을 검색한 것과 비교 (같은 결과)

In [11]:
body['query']['match']['name']['query'] = "패딩"
parsed_data = get_response(body)
print_result(parsed_data)

패딩
패딩조끼
디스커버리 패딩
남자 패딩
아이더 패딩
키즈 패딩
노스페이스 패딩
여성 패딩
남자 패딩코트
k2 패딩


<hr>
# 2) 성별, 나이대 지정

In [6]:
while True:
    gender = input("성별(f/m or 남/여): ")
    if gender=="여":
        gender = "f"
    elif gender=="남":
        gender = "m"
    if gender=="f" or gender=="m":
        break
    else:
        print("다시 입력해 주세요")

while True:
    age = input("나이(ex: 20세-->20): ")
    # 10세 미만일 시 10대와 동일시
    if int(age)<10:
        age = "10"
    # 50세 초과시 40대와 동일시
    elif int(age)>50:
        age = "40"
    if age.isnumeric():
        break
    else:
        print("숫자로 입력해주세요")

input_name = input("검색어를 입력하세요: ")



body = {
            "sort":[
                {"counts."+age[0]+gender:{"order":"desc"}},
                "_score"
            ],
            "size":10,
            "query": {
                "match": {
                  "name": {
                      "query": input_name,
                      "fuzziness":"AUTO"
                  }
                }
              }
            }

response = requests.get(URL, 
                        data= json.dumps(body),
                        headers={"Content-Type": "application/json"}) 

if(response.status_code==200):
    data = response.text
    parsed_data = json.loads(data)
    print("===============================")
    print_result(parsed_data)
else:
    print(response.status_code)
    print(response.json())

성별(f/m or 남/여): 여
나이(ex: 20세-->20): 24
검색어를 입력하세요: 쳌크
체크코트
체크스커트
체크원피스
체크자켓
체크 스커트
체크 핸드메이드 코트
체크 머플러
체크롱스커트
체크머플러
남자 체크 코트


In [5]:
while True:
    gender = input("성별(f/m or 남/여): ")
    if gender=="여":
        gender = "f"
    elif gender=="남":
        gender = "m"
    if gender=="f" or gender=="m":
        break
    else:
        print("다시 입력해 주세요")

while True:
    age = input("나이(ex: 20세-->20): ")
    # 10세 미만일 시 10대와 동일시
    if int(age)<10:
        age = "10"
    # 50세 초과시 40대와 동일시
    elif int(age)>50:
        age = "40"
    if age.isnumeric():
        break
    else:
        print("숫자로 입력해주세요")

input_name = input("검색어를 입력하세요: ")



body = {
            "sort":[
                {"counts."+age[0]+gender:{"order":"desc"}},
                "_score"
            ],
            "size":10,
            "query": {
                "match": {
                  "name": {
                      "query": input_name,
                      "fuzziness":"AUTO"
                  }
                }
              }
            }

response = requests.get(URL, 
                        data= json.dumps(body),
                        headers={"Content-Type": "application/json"}) 

if(response.status_code==200):
    data = response.text
    parsed_data = json.loads(data)
    print("===============================")
    print_result(parsed_data)
else:
    print(response.status_code)
    print(response.json())

성별(f/m or 남/여): 여
나이(ex: 20세-->20): 24
검색어를 입력하세요: 닡트
니트
니트원피스
니트 원피스
캐시미어 니트
니트조끼
니트 투피스
니트스커트
오프숄더 니트
꽈배기 니트
니트롱원피스
